# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine

import pickle
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier as clf
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

nltk.download(['punkt','wordnet','stopwords', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database

engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("SELECT * FROM InsertTableName", engine)
#df.genre.value_counts()


X = df.message
y = df.iloc[:,4:]
df.tail()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
26104,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26105,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26106,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26107,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
26108,30265,A radical shift in thinking came about as a re...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
# Checking the messages
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex, " ", text.lower().strip()) 
    words = word_tokenize(text)
    tokens = [WordNetLemmatizer().lemmatize(w) for w in words if w not in stopwords.words("english")]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf',clf(RandomForestClassifier()))
    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)
# Fitting the training set to the pipeline
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000021BB6DCB820>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Predicting on the test set
y_pred = pipeline.predict(X_test)

# printing the classification report
for i in range(len(y.axes[1])):
    print(y_test.columns[i], '-')
    
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

related -
              precision    recall  f1-score   support

           0       0.74      0.36      0.48      1547
           1       0.82      0.96      0.89      4938
           2       0.26      0.23      0.24        43

    accuracy                           0.81      6528
   macro avg       0.61      0.52      0.54      6528
weighted avg       0.80      0.81      0.79      6528

request -
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5420
           1       0.85      0.49      0.62      1108

    accuracy                           0.90      6528
   macro avg       0.87      0.73      0.78      6528
weighted avg       0.89      0.90      0.89      6528

offer -
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6503
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6528
   macro avg       0.50      0.50      0.50   

C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.93      1.00      0.97      6090
           1       0.25      0.00      0.00       438

    accuracy                           0.93      6528
   macro avg       0.59      0.50      0.48      6528
weighted avg       0.89      0.93      0.90      6528

transport -
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6208
           1       0.69      0.11      0.18       320

    accuracy                           0.95      6528
   macro avg       0.82      0.55      0.58      6528
weighted avg       0.94      0.95      0.94      6528

buildings -
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6190
           1       0.79      0.11      0.19       338

    accuracy                           0.95      6528
   macro avg       0.87      0.55      0.58      6528
weighted avg       0.94      0.95      0.93      6

C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {'clf__estimator__n_estimators':[10,50],
              'clf__estimator__min_samples_split':[2,3]
             }

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)
'''Note: Search reduced to a smaller range since the time to run becomes extremely high'''

'Note: Search reduced to a smaller range since the time to run becomes extremely high'

In [9]:
# printing the best parameters
print("\nBest Parameters:", cv.best_params_)
# Setting the model parameters to the best parameters
model = cv.best_estimator_


Best Parameters: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 50}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_pred= pd.DataFrame(model.predict(X_test), columns=y.columns)

In [11]:
y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
3478,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8764,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12965,1,1,0,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
1973,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
22846,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25813,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5731,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19543,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
y_pred
#labels = y.columns
#labels[2]
#labels=np.unique(y_pred.iloc[:,1])
#labels

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6523,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6524,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6525,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6526,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# printing the classification report

for i in range(36):
    print(y_test.columns[i], '-')
    
    print(classification_report(y_test.iloc[:,i], y_pred.iloc[:,i],target_names=y.columns)) #target_names=y.columns.values))
#print(classification_report(y_test, y_pred)) #, target_names=y.columns.values))
'''Note: Specifying the category names to target names outputs an error with the new sci-kit learn version. 
Will be considered in the next version'''

related -


ValueError: Number of classes, 3, does not match size of target_names, 36. Try specifying the labels parameter

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
 print(model)

### 9. Export your model as a pickle file

In [ ]:
#with open('classifier.pkl','wb') as file:
    #pickle.dump(cv, file)
    
pickle.dump(model, open('cv.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.